In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
# from toolbox import *
import toolbox as tb
from skeleton import *
import pandas as pd
from gensim.models import FastText
%load_ext autoreload
%autoreload 2
import os
import tqdm
from microtc.utils import tweet_iterator
import datetime
from collections import Counter
import gc
from collections import defaultdict

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
class Tokens():
    tokenized_pointer=None
    
    def tokenize(self,json_file,npy_file=None,replace=False,showProgressEach=1000):
        self.npy_file=str(os.path.splitext(json_file)[0]+".npy") if not npy_file else npy_file

        if not replace and os.path.isfile(self.npy_file):
            print(f"** Replace is off. {os.path.abspath(self.npy_file)} already exists, then load.")
        else:
            tokenized_docs=[]
            self.N=0
            print(f"** Processing {json_file} ...")
            tx = datetime.datetime.now()
            for idx, tw in enumerate(tweet_iterator(json_file)):
                tb.show_progress(showProgressEach, tx, idx)
                twTokens = tb.process_line(tw['text']) # Tokenizando tweet.
                tokenized_docs.append(twTokens)
            self.N=idx+1
        
            maxLen=len(max(tokenized_docs))
            for i, doc in enumerate(tokenized_docs):
                tokenized_docs[i]=[doc[x] if x<len(doc) else '' for x in range(maxLen)]
            
            np.save(self.npy_file,tokenized_docs)
            del(tokenized_docs)
            gc.collect()
            print(f"** Processed {self.N} lines. Saved to {os.path.abspath(self.npy_file)}.")
        
        self.pointer=np.load(self.npy_file, mmap_mode='r')
        return self
    
    def get(self,n):
        return [x for x in self.pointer[n,:] if x != '']
    
    def getDocs(self,docs):
#         if isinstance(docs,int): docs=[docs]
        res=[]
        for n in docs:
            res.append(self.get(n))
        return res

In [13]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:16:27.990884 :: 0.000193 - Processing item #0 
** Processed 10000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [24]:
class Index():
    N = 0
    
    def computePostingLists(self, tokens, showProgressEach=1000):
        tx = datetime.datetime.now()
        DF = Counter({})
        TF = []
        
        print("** Counting TF & DF ...")
        for idx in range(tokens.N):
            tb.show_progress(showProgressEach,tx,idx)
            tok=tokens.get(idx)
            twCnt = Counter(tok) # Sumando frecuencias individiales.
            TF.append(twCnt)
            DF.update(list(twCnt.keys())) # Sumando frecuencias por documento (DF).
            
        self.N=idx+1 # Contabilizando documentos.
        self.postlists=defaultdict(list)
        print("** Calculating TF-IDF...")
        for i,c in enumerate(TF):
            tb.show_progress(showProgressEach,tx,i)
            for w,tf in c.items():
                self.postlists[w].append((tf*np.log2(self.N/(DF[w]+1)),i))
        del(TF)
        del(DF)
        gc.collect()
        
        print("** Sorting posting lists...")
        for idx,(w,p) in enumerate(self.postlists.items()):
            tb.show_progress(showProgressEach,tx,idx)
            p.sort()
            p=list(zip(*p))
            self.postlists[w]=[list(p[1]),list(p[0])]
        
        return self
    
    def getIdxs(self,word):
        return self.postlists[word][0]
    
    def getScores(self,word):
        return self.postlists[word][1]
    
    def a(self, q, isText=False):
        """Búsqueda AND."""
        if isText: q=tb.process_line(q)
        res = None
        for word in q:
            if not res:
                res = set(self.getIdxs(word))
            else:
                res = res.intersection(self.getIdxs(word))
        return list(sorted(res))

    def o(self, q, isText=False):
        """Búsqueda OR."""
        if isText: q=tb.process_line(q)
        res = set()
        for word in q:
            res = res.union(self.getIdxs(word))
        return list(sorted(res))    

In [25]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
# index_.postlists

** Counting TF & DF ...
2020-05-12 19:18:55.281691 :: 0.001182 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:18:55.935591 :: 0.655082 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:18:56.539612 :: 1.259103 - Processing item #0 
2020-05-12 19:18:56.634560 :: 1.354051 - Processing item #10000 


In [26]:
len(index_.postlists.keys())

10383

In [33]:
ds=index_.o('hoy no me puedo levantar',isText=True)
print(len(ds),ds[:5])
docs=tokens.getDocs(ds)
print(len(docs),docs[:5])

619 [3, 5, 9, 45, 54]
619 [['obvi', 'hoy', 'cumpl', 'anos', 'apag', 'voz'], ['hoy', 'cumpl', 'ano', 'plan', 'des', 'secret', 'gent', 'sabi', 'haci', 'sent', 'bien', 'aunqu', 've', 'tan', 'cercan'], ['amig', 'seguidor', 'tod', 'person', 'respet', 'jueg', 'decision', 'jug', 'pued', 'influ', 'ser', 'president', 'estudiantil', 'hag', 'ayud'], ['pregunt', 'quer', 'hoy'], ['pregunt', 'investig', 'aprend', 'pued']]


In [ ]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)